In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle # Import pickle here as it's used later
import seaborn as sns # Import seaborn explicitly
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc # Import metrics here

# Cargar el dataset
data = pd.read_csv('UCI_Credit_Card.csv')

# Exploración inicial
print(data.head())
print(data.info())
print(data.describe())

# Verificar valores nulos
print(data.isnull().sum())

# Eliminar duplicados si existen
data = data.drop_duplicates()

# Separar características y variable objetivo
# X originally has 23 features ('ID' dropped)
X = data.drop(['default.payment.next.month', 'ID'], axis=1) # Ensure 'ID' is dropped here
y = data['default.payment.next.month']

# Codificar variables categóricas si las hay (should not be any in this dataset)
# Although not strictly needed for this dataset, keep the code for generality
categorical_cols = X.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    X = pd.get_dummies(X, columns=categorical_cols)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las características
# IMPORTANT: Fit scaler *before* adding the intercept
scaler = StandardScaler()
print(f"Shape of X_train before fitting scaler: {X_train.shape}") # Debug print
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Añadir término de intercepción (bias) *after* scaling
X_train = np.hstack([np.ones((X_train_scaled.shape[0], 1)), X_train_scaled])
X_test = np.hstack([np.ones((X_test_scaled.shape[0], 1)), X_test_scaled])

# Verify shapes after adding intercept
print(f"Shape of X_train after adding intercept: {X_train.shape}") # Debug print
print(f"Shape of X_test after adding intercept: {X_test.shape}")   # Debug print


class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-4):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.weights = None
        self.loss_history = [] # Store loss for each fit

    def sigmoid(self, z):
        # Handle potential overflow/underflow in sigmoid
        z = np.clip(z, -500, 500) # Clip values to prevent exp overflow
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, X, y, weights):
        m = X.shape[0]
        h = self.sigmoid(X @ weights)
        # Add epsilon to log arguments to prevent log(0)
        epsilon = 1e-15
        # Use binary cross-entropy
        loss = (-1/m) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1-h + epsilon))
        return loss

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.loss_history = [] # Initialize loss history for the current fit

        # Calculate initial loss before training
        self.loss_history.append(self.compute_loss(X, y, self.weights))


        for i in range(self.max_iter):
            # Calcular predicciones
            h = self.sigmoid(X @ self.weights)

            # Calcular gradiente
            gradient = (1/m) * (X.T @ (h - y))

            # Actualizar pesos
            new_weights = self.weights - self.learning_rate * gradient

            # Calculate loss after weight update (only if you want per-iteration loss)
            # current_loss = self.compute_loss(X, y, new_weights)
            # self.loss_history.append(current_loss)

            # Verificar convergencia usando el gradiente
            if np.linalg.norm(gradient) < self.tol:
                 print(f"Convergencia basada en gradiente alcanzada en la iteración {i+1}")
                 self.weights = new_weights # Update weights one last time
                 # If loss history was not recorded per iteration, calculate final loss here
                 # final_loss = self.compute_loss(X, y, self.weights)
                 # self.loss_history.append(final_loss)
                 break

            self.weights = new_weights

        # Always record the final loss after the loop finishes (either by max_iter or convergence)
        final_loss = self.compute_loss(X, y, self.weights)
        self.loss_history.append(final_loss) # Ensure final loss is recorded


        return self

    def predict_proba(self, X):
        # Ensure input X has the correct shape (including intercept)
        if X.shape[1] != self.weights.shape[0]:
             raise ValueError(f"Input shape mismatch in predict_proba: Expected {self.weights.shape[0]} features (including intercept), but got {X.shape[1]}.")
        return self.sigmoid(X @ self.weights)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

class MultiClassLogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-4):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.classifiers = []
        self.classes = None # Initialize classes here

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.classifiers = [] # Clear classifiers list before refitting

        # Ensure input X has the correct shape (including intercept)
        # All binary classifiers will be trained on X with intercept
        expected_features = X.shape[1]


        for cls in self.classes:
            print(f"Training classifier for class: {cls}") # Debug print

            # Crear etiquetas binarias para la clase actual
            y_binary = (y == cls).astype(int)

            # Entrenar clasificador binario
            lr = LogisticRegression(learning_rate=self.learning_rate,
                                  max_iter=self.max_iter,
                                  tol=self.tol)
            lr.fit(X, y_binary) # X here already includes the intercept

            # Verify fitted weights shape
            if lr.weights.shape[0] != expected_features:
                raise ValueError(f"Fitted weights shape mismatch for class {cls}: Expected {expected_features}, but got {lr.weights.shape[0]}.")

            self.classifiers.append(lr)

        return self

    def predict_proba(self, X):
        # Ensure input X has the correct shape (including intercept)
        if not self.classifiers:
             raise RuntimeError("Model has not been fitted yet.")
        if X.shape[1] != self.classifiers[0].weights.shape[0]:
             raise ValueError(f"Input shape mismatch in predict_proba: Expected {self.classifiers[0].weights.shape[0]} features (including intercept), but got {X.shape[1]}.")

        probas = np.array([clf.predict_proba(X) for clf in self.classifiers]).T
        # Normalizar para que sumen 1 (important for multi-class interpretation)
        probas_sum = probas.sum(axis=1, keepdims=True)
        # Handle case where sum is zero to avoid division by zero
        probas_sum[probas_sum == 0] = 1 # Or a small epsilon
        return probas / probas_sum


    def predict(self, X):
        probas = self.predict_proba(X)
        # Ensure classes are defined before using them as index
        if self.classes is None:
            raise RuntimeError("Model has not been fitted yet. Classes are not defined.")
        return self.classes[np.argmax(probas, axis=1)]

# Crear y entrenar el modelo
# The model expects X_train which already includes the intercept and is scaled
model = MultiClassLogisticRegression(learning_rate=0.1, max_iter=1000)
model.fit(X_train, y_train) # Use X_train with intercept and scaled features

# --- Generate and Save Plots ---

# Graficar la pérdida durante el entrenamiento (para el primer clasificador)
print("\nGenerating and Saving Loss Plot...")
if model.classifiers and model.classifiers[0].loss_history:
    plt.figure(figsize=(10, 6))
    # Only plot if loss history was recorded.
    plt.plot(model.classifiers[0].loss_history)
    plt.title('Pérdida durante el entrenamiento (Primer clasificador)')
    plt.xlabel('Iteración')
    plt.ylabel('Pérdida')
    plt.grid()
    plt.savefig('loss_plot.png') # Save the plot for Gradio
    plt.show() # Show in notebook
else:
    print("No loss history recorded or classifiers not trained for loss plot. Skipping save and display.")
plt.close('all') # Close all figures to free memory

# Calculate metrics before generating plots
# Predecir en el conjunto de prueba
# model.predict expects X_test which already includes the intercept and is scaled
y_pred = model.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrecisión: {accuracy:.4f}")

# Matriz de confusión
print("\nMatriz de confusión:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


# Matriz de confusión visual
print("\nGenerating and Saving Confusion Matrix Plot...")
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=model.classes, yticklabels=model.classes)
plt.title('Matriz de Confusión')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.savefig('confusion_matrix_plot.png') # Save the plot for Gradio
plt.show() # Show in notebook
plt.close('all') # Close all figures to free memory


# Curvas ROC (para clasificación binaria)
print("\nGenerating and Saving ROC Curve (if binary)...")
if len(model.classes) == 2:
    # y_proba needs to be probabilities for the positive class (usually class 1)
    # Find the index for class 1
    class_1_idx = np.where(model.classes == 1)[0]
    if class_1_idx.size > 0:
        class_1_idx = class_1_idx[0]
        y_proba = model.predict_proba(X_test)[:, class_1_idx] # Get probabilities for class 1
        fpr, tpr, thresholds = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Curva ROC')
        plt.legend(loc="lower right")
        plt.savefig('roc_curve_plot.png') # Save the plot for Gradio
        plt.show() # Show in notebook
    else:
        print("Class 1 not found in model classes for ROC curve calculation. Skipping ROC plot save and display.")
else:
    print("Skipping ROC curve: Not a binary classification problem.")
plt.close('all') # Close all figures to free memory


# Mostrar los coeficientes más importantes for print, not plot in Gradio
print("\nShowing top coefficients per class (printed to console)...")
# Ensure feature_names includes the intercept and excludes 'ID'
feature_names_with_intercept = ['Intercept'] + list(data.drop(['default.payment.next.month', 'ID'], axis=1).columns)

for i, cls in enumerate(model.classes):
    print(f"\nCoeficientes para la clase {cls}:")
    if i < len(model.classifiers): # Check if classifier exists for this index
        coef_df = pd.DataFrame({
            'Feature': feature_names_with_intercept, # Use the feature names with intercept
            'Coefficient': model.classifiers[i].weights # Access weights from the specific classifier
        })
        # Sort by absolute value of coefficient
        coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
        coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False).drop('Abs_Coefficient', axis=1)

        print(coef_df.head(10))
    else:
        print(f"Classifier for class {cls} not found.")


# Guardar el modelo entrenado
with open('credit_card_default_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Guardar el scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\nModel and Scaler saved.")

In [ ]:
!pip install pandas numpy scikit-learn gradio matplotlib seaborn

import gradio as gr
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os # Import os module to check for files

# 1. Cargar modelo y scaler
try:
    with open('credit_card_default_model.pkl', 'rb') as f:
        model = pickle.load(f)

    with open('scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    model_load_status = "Modelo y Scaler cargados correctamente."
except FileNotFoundError:
     model_load_status = "Error: Asegúrate de ejecutar la primera celda para entrenar y guardar el modelo y scaler."
     model = None # Set model to None if loading fails
     scaler = None # Set scaler to None if loading fails


# 2. Verificación crítica (only if model loaded successfully)
if model:
    try:
        print("Features esperadas por el scaler (sin intercepto):", scaler.n_features_in_)  # Debe ser 23
        print("Features entrenadas por el modelo (con intercepto):", model.classifiers[0].weights.shape[0]) # Debe ser 24
    except Exception as e:
        print(f"Error durante la verificación del modelo/scaler: {e}")


# 3. Definir TODAS las características (24 features)
# Nota: Agregué 'INTERCEPT' como primer feature para coincidir con el scaler y el modelo entrenado
feature_names = [
    'INTERCEPT',  # ¡Nueva! Esto explica los 24 features
    'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
    'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
    'BILL_AMT5', 'BILL_AMT6',
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',
    'PAY_AMT5', 'PAY_AMT6'
]

# 4. Función para preparar inputs (8 → 24)
def prepare_input(*input_values):
    """Convierte 8 inputs en array de 24 features (con intercepción)"""
    # Valores por defecto para los features no proporcionados (ajustar según necesidad)
    # Es crucial que estos valores por defecto estén en el mismo rango o escala que los datos de entrenamiento
    # antes de la estandarización, aunque luego se escalarán. Usar 0 para los pagos/facturas
    # retrasados no parece incorrecto si representa "sin retraso/factura".
    default_values = {
        'PAY_2': 0, 'PAY_3': 0, 'PAY_4': 0, 'PAY_5': 0, 'PAY_6': 0,
        'BILL_AMT2': 0, 'BILL_AMT3': 0, 'BILL_AMT4': 0, 'BILL_AMT5': 0, 'BILL_AMT6': 0,
        'PAY_AMT2': 0, 'PAY_AMT3': 0, 'PAY_AMT4': 0, 'PAY_AMT5': 0, 'PAY_AMT6': 0
    }

    # Mapear inputs proporcionados a sus nombres de característica
    input_dict = {
        'LIMIT_BAL': input_values[0],
        'SEX': input_values[1],
        'EDUCATION': input_values[2],
        'MARRIAGE': input_values[3],
        'AGE': input_values[4],
        'PAY_0': input_values[5], # Assuming PAY_0 is the 'ÚLTIMO PAGO' input
        'BILL_AMT1': input_values[6], # Assuming BILL_AMT1 is the 'MONTO FACTURA' input
        'PAY_AMT1': input_values[7] # Assuming PAY_AMT1 is the 'ÚLTIMO PAGO REALIZADO' input
    }

    # Combinar inputs proporcionados con valores por defecto
    full_features = {**default_values, **input_dict}

    # Crear el array de features en el orden correcto, incluyendo el intercepto (1.0)
    # Usamos feature_names para asegurar el orden
    input_array = np.array([1.0] + [full_features[col] for col in feature_names[1:]])

    return input_array

# 5. Función de predicción
def predict_credit_default(*input_values):
    # Check if model and scaler were loaded successfully
    if model is None or scaler is None:
        return model_load_status, None # Return the error message

    try:
        # 1. Preparar input (array de 24 features, con intercepto incluido)
        prepared_input = prepare_input(*input_values).reshape(1, -1)
        # print("Input shape después de prepare_input:", prepared_input.shape) # Debug print (shape should be (1, 24))

        # 2. Estandarizar las características (excluir el intercepto en la columna 0)
        # El scaler fue entrenado en los datos SIN intercepto.
        # Por lo tanto, debemos aplicar la transformación solo a las columnas 1 en adelante.
        prepared_input_scaled = prepared_input.copy() # Crear una copia para no modificar el original antes de escalar
        prepared_input_scaled[:, 1:] = scaler.transform(prepared_input[:, 1:])
        # print("Input shape después de escalar:", prepared_input_scaled.shape) # Debug print (shape should be (1, 24))

        # 3. Predecir
        # El modelo espera el array con el intercepto ya incluido y las demás features escaladas.
        prediction = model.predict(prepared_input_scaled)[0]
        proba = model.predict_proba(prepared_input_scaled)[0]

        # --- DEBUG PRINT: Check if probabilities change ---
        print(f"Prediction Probabilities: {proba}")
        # --------------------------------------------------

        # 4. Visualización de probabilidad para la predicción actual
        print("Generating individual prediction plot...") # Debug print before plotting
        fig, ax = plt.subplots(figsize=(6, 4))
        # Ensure classes exist and are used correctly for plotting
        if model.classes is not None and len(model.classes) == len(proba):
             ax.bar(model.classes.astype(str), proba, color=['green', 'red']) # Usar las clases reales
             ax.set_xticks(model.classes) # Asegurar que los ticks del eje x coincidan con las clases
             ax.set_xticklabels(['No Default', 'Default']) # Etiquetas legibles
             ax.set_ylabel("Probabilidad")
             ax.set_ylim(0, 1) # Limitar el eje Y entre 0 y 1
        else:
            ax.text(0.5, 0.5, "Could not plot probabilities", horizontalalignment='center', verticalalignment='center')

        plt.close(fig)

        # 5. Formato de salida
        # Asegurarse de que las probabilidades corresponden a las clases 0 (No Default) y 1 (Default)
        # model.classes[0] will be 0 and model.classes[1] will be 1 if they are ordered
        # Use np.where to find indices just in case
        proba_no_default = proba[np.where(model.classes == 0)[0][0]] if model.classes is not None and 0 in model.classes else 0
        proba_default = proba[np.where(model.classes == 1)[0][0]] if model.classes is not None and 1 in model.classes else 0


        return (
            f"Predicción: {'Default ⚠️' if prediction == 1 else 'No Default ✅'}\n"
            f"Probabilidad Default: {proba_default*100:.2f}%\n"
            f"Probabilidad No Default: {proba_no_default*100:.2f}%",
            fig
        )

    except Exception as e:
        # print("Error details:", e) # Debug print
        return f"❌ Error durante la predicción: {str(e)}", None

# 6. Interfaz Gradio
# Define the paths to the saved plot images
LOSS_PLOT_PATH = 'loss_plot.png'
CONFUSION_MATRIX_PLOT_PATH = 'confusion_matrix_plot.png'
ROC_CURVE_PLOT_PATH = 'roc_curve_plot.png'

# Check if plot files exist
loss_plot_exists = os.path.exists(LOSS_PLOT_PATH)
cm_plot_exists = os.path.exists(CONFUSION_MATRIX_PLOT_PATH)
roc_plot_exists = os.path.exists(ROC_CURVE_PLOT_PATH)


# Define inputs (same as before)
inputs = [
    gr.Number(label="LÍMITE DE CRÉDITO (LIMIT_BAL)", value=200000),
    gr.Dropdown(label="GÉNERO (SEX)", choices=[("Masculino", 1), ("Femenino", 2)], value=1),
    gr.Dropdown(label="EDUCACIÓN (EDUCATION)",
               choices=[("Posgrado", 1), ("Universidad", 2), ("Preparatoria", 3), ("Otros", 4), ("(Desconocido)", 0), ("(Desconocido)", 5), ("(Desconocido)", 6)], # Incluir todas las opciones posibles según el dataset
               value=2),
    gr.Dropdown(label="ESTADO CIVIL (MARRIAGE)",
               choices=[("Casado", 1), ("Soltero", 2), ("Otros", 3), ("(Desconocido)", 0)], # Incluir todas las opciones posibles
               value=1),
    gr.Number(label="EDAD (AGE)", value=30),
    gr.Dropdown(label="ESTADO DE PAGO (Último Mes - PAY_0)",
               choices=[("Puntual", 0), ("Retraso 1 mes", 1), ("Retraso 2 meses", 2),
                        ("Retraso 3 meses", 3), ("Retraso 4 meses", 4),
                        ("Retraso 5 meses", 5), ("Retraso 6 meses", 6),
                        ("Retraso 7 meses", 7), ("Retraso 8 meses", 8),
                        ("Retraso 9+ meses", 9), ("Adelantado", -1), ("Adelantado", -2)], # Incluir todas las opciones posibles
               value=0),
    gr.Number(label="MONTO FACTURA (Último Mes - BILL_AMT1)", value=5000),
    gr.Number(label="ÚLTIMO PAGO REALIZADO (PAY_AMT1)", value=2000),
]

# Define outputs - add Image components for the saved plots
outputs = [
    gr.Textbox(label="Resultado de Predicción"),
    gr.Plot(label="Probabilidad de Default (Predicción Individual)"),
    # Add Image outputs for training/evaluation plots
    gr.Image(label="Pérdida durante el Entrenamiento", visible=loss_plot_exists) if loss_plot_exists else gr.Textbox(label="Pérdida durante el Entrenamiento", value="Gráfica no disponible (ejecute la primera celda)", visible=True),
    gr.Image(label="Matriz de Confusión (Test Set)", visible=cm_plot_exists) if cm_plot_exists else gr.Textbox(label="Matriz de Confusión (Test Set)", value="Gráfica no disponible (ejecute la primera celda)", visible=True),
    gr.Image(label="Curva ROC (Test Set, si es Binario)", visible=roc_plot_exists) if roc_plot_exists else gr.Textbox(label="Curva ROC (Test Set, si es Binario)", value="Gráfica no disponible (ejecute la primera celda)", visible=True)
]

# Update the fn function to also return the plot images/placeholders
def full_interface_fn(*input_values):
    # Call the prediction function first
    prediction_output_text, prediction_output_plot = predict_credit_default(*input_values)

    # Load the saved plot images if they exist
    loss_plot_img = LOSS_PLOT_PATH if os.path.exists(LOSS_PLOT_PATH) else None
    cm_plot_img = CONFUSION_MATRIX_PLOT_PATH if os.path.exists(CONFUSION_MATRIX_PLOT_PATH) else None
    roc_plot_img = ROC_CURVE_PLOT_PATH if os.path.exists(ROC_CURVE_PLOT_PATH) else None

    # Return all outputs in the order defined in the outputs list
    # Need to return the prediction_output_plot here
    return prediction_output_text, prediction_output_plot, loss_plot_img, cm_plot_img, roc_plot_img


iface = gr.Interface(
    fn=full_interface_fn, # Use the new function that returns all outputs
    inputs=inputs,
    outputs=outputs, # Use the updated outputs list
    title="🔍 Predictor de Riesgo Crediticio y Evaluación del Modelo",
    description="Complete los 8 campos clave para obtener una predicción individual. También se muestran gráficas de evaluación del modelo entrenado.",
    examples=[
        [200000, 1, 2, 1, 30, 0, 5000, 2000],
        [50000, 2, 3, 2, 45, 2, 25000, 500],
        [10000, 1, 1, 1, 25, -1, 1000, 5000], # Example with early payment
        [300000, 2, 2, 2, 50, 3, 100000, 0] # Example with significant delay
    ]
)

iface.launch()